In [1]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
import requests
from bs4 import BeautifulSoup

# =========================
# Simple conversation memory
# =========================
class SimpleMemory:
    def __init__(self):
        self.history = []

    def add(self, role, message):
        self.history.append(f"{role}: {message}")

    def get_context(self):
        # Join past messages as a string context
        return "\n".join(self.history)

memory = SimpleMemory()

# =========================
# Initialize Ollama LLM
# =========================
llm = OllamaLLM(model="llama3.1", temperature=0)

# =========================
# Tool 1: QA Tool
def qa_tool_func(question):
    context = memory.get_context()
    prompt_text = f"{context}\nUser Question: {question}\nAnswer clearly:"
    response = llm.invoke(prompt_text)  # CRT fix: pass string, not dict
    memory.add("User", question)
    memory.add("AI", response)
    return response

qa_tool = Tool(
    name="QA Tool",
    func=qa_tool_func,
    description="Answer factual questions clearly using memory"
)

# =========================
# Tool 2: Web Search Tool
def web_search_tool(query):
    url = f"https://www.google.com/search?q={query}"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    results = []
    for g in soup.find_all('div', class_='tF2Cxc')[:3]:
        title = g.find('h3').text if g.find('h3') else ""
        link = g.find('a')['href'] if g.find('a') else ""
        results.append(f"{title} - {link}")
    return "\n".join(results)

search_tool = Tool(
    name="Web Search",
    func=web_search_tool,
    description="Search Google without API key"
)

# =========================
# Tools dictionary
# =========================
tools = {
    "QA": qa_tool,
    "Web Search": search_tool
}

# =========================
# Run queries manually
# =========================
queries = [
    ("QA", "What is LangGraph in LangChain?"),
    ("Web Search", "Latest updates on GPT-4"),
    ("QA", "Explain LangChain memory in brief")
]

for tool_name, user_input in queries:
    print(f"\n🧑‍💻 Tool: {tool_name}")
    print(f"Input: {user_input}")
    output = tools[tool_name].run(user_input)
    print(f"\n🤖 Output:\n{output}")



🧑‍💻 Tool: QA
Input: What is LangGraph in LangChain?

🤖 Output:
LangChain is an open-source library for building conversational AI applications. LangGraph is a component of LangChain that enables the creation and manipulation of graph-like data structures, specifically designed to represent complex relationships between entities and concepts.

In essence, LangGraph allows developers to build and query knowledge graphs, which are graphical representations of entities, their attributes, and the relationships between them. This feature is particularly useful for applications like question answering, entity disambiguation, and conversational dialogue management.

LangGraph provides a flexible and scalable way to represent and reason about complex knowledge graphs, making it an essential tool for building sophisticated conversational AI systems with LangChain.

🧑‍💻 Tool: Web Search
Input: Latest updates on GPT-4

🤖 Output:


🧑‍💻 Tool: QA
Input: Explain LangChain memory in brief

🤖 Output:
*

In [10]:
query = "Summarize LangChain in 2 lines, then tell me who created it."
print("Query:", query)
print("Response:", qa_tool.run(query))


Query: Summarize LangChain in 2 lines, then tell me who created it.
Response: **Summary of LangChain:**

LangChain is an open-source library for building conversational AI applications. It provides a set of components, including LangGraph and memory, to enable the creation of sophisticated conversational AI systems.

**Creator of LangChain:**
LangChain was created by Meta AI.
